# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [45]:
#Your code here
import pandas as pd
import numpy as np
df = pd.read_csv('homepage_actions.csv')
# Let's group this dataframe by the experimental group each point is in, and count each groups actions.
dfGrouped = df.groupby('group')['action'].value_counts()
# Now, let's normalize these to find two separate click rates, one for each group.
dfGrouped
controlGroupSize = dfGrouped['control'][0]+dfGrouped['control'][1]
experimentGroupSize = dfGrouped['experiment'][0]+dfGrouped['experiment'][1]
controlClickRate = dfGrouped['control'][1]/controlGroupSize
expClickRate = dfGrouped['experiment'][1]/experimentGroupSize
print(f"Control Group Success: {controlClickRate}. Experimental Group Success: {expClickRate}, Control Sample Size: {controlGroupSize}, Experiment Sample Size = {experimentGroupSize}")

Control Group Success: 0.21857410881801126. Experimental Group Success: 0.23649337410805302, Control Sample Size: 4264, Experiment Sample Size = 3924


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [42]:
#Your code here
# Let's normalize these to find two separate click rates, one for each group.
dfGrouped
controlGroupSize = dfGrouped['control'][0]+dfGrouped['control'][1]
experimentGroupSize = dfGrouped['experiment'][0]+dfGrouped['experiment'][1]
controlClickRate = dfGrouped['control'][1]/dfGrouped['control'][0]
expClickRate = dfGrouped['experiment'][1]/dfGrouped['experiment'][0]
print(f"Control Group Success: {controlClickRate}. Experimental Group Success: {expClickRate}, Control Sample Size: {controlGroupSize}, Experiment Sample Size = {experimentGroupSize}")

import statsmodels.stats as stats
# Ho: Their is no difference between the two groups
# Ha: Their is a difference between the two groups.
alpha = 0.05
stats.proportion.proportions_ztest([controlClickRate * controlGroupSize, expClickRate * experimentGroupSize], [controlGroupSize, experimentGroupSize], alternative='two-sided')
#if z_test_result.pvalue < alpha:
#    print(f'Reject the null hypothesis with p-value {z_test_result.pvalue:.4f} < alpha = {alpha}')
#else:
#    print(f'Fail to reject the null hypothesis with p-value {z_test_result.pvalue:.4f} >= alpha = {alpha}')

print(f'Given our p-value of 0.00288, which is less than our alpha of 0.05, we reject the null hypothesis.')


Control Group Success: 0.2797118847539016. Experimental Group Success: 0.3097463284379172, Control Sample Size: 4264, Experiment Sample Size = 3924
Given our p-value of 0.00288, which is less than our alpha of 0.05, we reject the null hypothesis.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [43]:
#Your code here
expected_experiment_clicks_null = controlClickRate*experimentGroupSize

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [49]:
#Your code here
var = experimentGroupSize*controlClickRate*(1-controlClickRate)
std = np.sqrt(var)
std

25.888551746650283

In [51]:
experiment_clicks = dfGrouped['experiment'][1]
zscore = (experiment_clicks - expected_experiment_clicks_null)/std
zscore

-6.550750209356649

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [55]:
#Your code here
import scipy.stats as spicy
pval = 1- spicy.norm.sf(zscore)
pval

2.86243251323981e-11

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes. That is a tiny p value which corresponds with our previous test.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.